In [61]:
from tfx import v1 as tfx
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import Transform
from tfx.components import Trainer
from tfx.orchestration.experimental.interactive.interactive_context import \
    InteractiveContext

In [74]:
import os
from pathlib import Path

from tfx.components import CsvExampleGen
from tfx.proto import example_gen_pb2

context = InteractiveContext()

dir_path = Path().parent.absolute()
data_dir = os.path.join(dir_path, "..", "..", "data", "processed")
output = example_gen_pb2.Output(
    # 선호하는 분할을 정의합니다.
    split_config=example_gen_pb2.SplitConfig(splits=[
        # 비율을 지정합니다.
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=6),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
        example_gen_pb2.SplitConfig.Split(name="test", hash_buckets=2)
    ]))

# output_config 인수를 추가합니다.
example_gen = CsvExampleGen(input_base=data_dir, output_config=output)
context.run(example_gen)

for artifact in example_gen.outputs["examples"].get():
    print(artifact)

Artifact(artifact: id: 1
type_id: 14
uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/CsvExampleGen/examples/1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\", \"test\"]"
  }
}
custom_properties {
  key: "file_format"
  value {
    string_value: "tfrecords_gzip"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:159954922,xor_checksum:1630270091,sum_checksum:1630270091"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.2.0"
  }
}
state: LIVE
, artifact_type: id: 14
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_n

In [75]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"])
context.run(statistics_gen)

context.show(statistics_gen.outputs["statistics"])

In [76]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"])
context.run(statistics_gen)

for artifact in statistics_gen.outputs["statistics"].get():
    print(artifact.uri)

/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/StatisticsGen/statistics/2


In [77]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"],
    infer_feature_shape=False,
)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 4
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/SchemaGen/schema/4"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [82]:
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath("transform.py"))
context.run(transform)

Exception ignored in: <function CapturableResource.__del__ at 0x7fb5c84029d0>
Traceback (most recent call last):
  File "/Users/chris/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py", line 277, in __del__
    self._destroy_resource()
  File "/Users/chris/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 889, in __call__
    result = self._call(*args, **kwds)
  File "/Users/chris/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 924, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/Users/chris/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/function.py", line 3022, in __call__
    filtered_flat_args) = self._maybe_define_function(args, kwargs)
  File "/Users/chris/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/function.py", line 3444, in _maybe_define_function
    graph_function = self._create_graph_function(args, kw

    graph_function = self._create_graph_function(args, kwargs)
  File "/Users/chris/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/function.py", line 3279, in _create_graph_function
    func_graph_module.func_graph_from_py_func(
  File "/Users/chris/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/framework/func_graph.py", line 999, in func_graph_from_py_func
    func_outputs = python_func(*func_args, **func_kwargs)
  File "/Users/chris/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 672, in wrapped_fn
    out = weak_wrapped_fn().__wrapped__(*args, **kwds)
AttributeError: 'NoneType' object has no attribute '__wrapped__'
Exception ignored in: <function CapturableResource.__del__ at 0x7fb5c84029d0>
Traceback (most recent call last):
  File "/Users/chris/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py", line 277, in __del__
    self._destroy_resource()
  File "/Users/chris/opt/

INFO:tensorflow:Assets written to: /var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/Transform/transform_graph/9/.temp_path/tftransform_tmp/f809278dfc834ea99e2db683066e82d4/assets


INFO:tensorflow:Assets written to: /var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/Transform/transform_graph/9/.temp_path/tftransform_tmp/f809278dfc834ea99e2db683066e82d4/assets


INFO:tensorflow:Assets written to: /var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/Transform/transform_graph/9/.temp_path/tftransform_tmp/67ce2cd62b654c60b75c34ace705dd4c/assets


INFO:tensorflow:Assets written to: /var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/Transform/transform_graph/9/.temp_path/tftransform_tmp/67ce2cd62b654c60b75c34ace705dd4c/assets


ExecutionResult(
    component_id: Transform
    execution_id: 9
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 18
        type_id: 20
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/Transform/transform_graph/9"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 19
        type_id: 14
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/Transform/transformed_examples/9"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\", \"test\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 20
        type_id: 21
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/Transform/updated_analyzer_cache/9"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 21
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 21
        type_id: 18
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/Transform/pre_transform_schema/9"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_compon

In [81]:
trainer = Trainer(
    module_file=os.path.abspath("trainer.py"),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    train_args=tfx.proto.TrainArgs(num_steps=100),
    eval_args=tfx.proto.EvalArgs(num_steps=50))
context.run(trainer)

1/1 [==============================] - ETA: 0s - loss: 0.6804 - binary_accuracy: 0.6000 - true_positives_2: 10.000 - 3s 3s/step - loss: 0.6804 - binary_accuracy: 0.6000 - true_positives_2: 10.0000 - val_loss: 0.6646 - val_binary_accuracy: 0.7100 - val_true_positives_2: 3.0000
INFO:tensorflow:Assets written to: /var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/Trainer/model/8/Format-Serving/assets


INFO:tensorflow:Assets written to: /var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/Trainer/model/8/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 8
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 16
        type_id: 24
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/Trainer/model/8"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 17
        type_id: 25
        uri: "/var/folders/rm/yb4phhx53p59nxyws_sr42yr0000gn/T/tfx-interactive-2021-08-30T05_48_28.250494-m5yq7r45/Trainer/model_run/8"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 25
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [99]:
import tensorflow as tf
model_path = trainer.outputs["model"].get()[0].uri + "/Format-Serving"
model = tf.saved_model.load(export_dir=model_path)
predict_fn = model.signatures["serving_default"]